Installing the requirements:

In [1]:
!pip3 install numpy

Defaulting to user installation because normal site-packages is not writeable


Importing needed libraries

In [14]:
import math
import random
import time
import numpy as np

Defining the class and algorithmm of RBF network

For calculating the centers and the sizes we used input data driven.

In [296]:
class RBF_net:
    #first we initialize the network and set number of neurones in each layer
    def __init__(self,Ni,Ki,Mi,flag_bias):
        self.eta=0.2
        self.N=Ni
        self.K=Ki
        self.M=Mi        
        self.ck=[random.random() for x in range(Ki)]
        self.sk=[0 for x in range(Ki)]
        self.rk=[0 for x in range(Ki)]
        self.om=[0 for x in range(Mi)]
        self.netm=[0 for x in range(Mi)]
        self.flag_bias=flag_bias
        if flag_bias:
            self.ws=[[(random.random()-0.5) for j in range(Ki+1)] for i in range(M)]
        else:
            self.ws=[[(random.random()-0.5) for j in range(Ki)] for i in range(Mi)]
        
        self.trans=math.tanh
        self.dtrans=lambda x: 1-x**2
        
    def distance(self,x,c):
        #this is the function to calculate the distance from a given input to the center then we power it to 2 and sum all of them to calculate the
        # the distance in higher dimensions 
        return abs(x-c)
    def radialFunction(self,dis,size):
        #for the radial function we use the gausian function
        return math.e**(-(dis**2)/(2*(size**2)))

    def forward(self,X):
        #one step of forwarding a given input throgh the network
        for k in range(self.K):
            #RBF layer
            dis=0
            for n in range(self.N):
                dis+=(self.distance(X[n],self.ck[k]))**2
            self.rk[k]=self.radialFunction(dis,self.sk[k])**(1/2)      

        for m in range(self.M):
            #output layer
            temp_sum=0
            for k in range(self.K):
               
                temp_sum+=self.rk[k]*self.ws[m][k] 
            if(self.flag_bias):
                temp_sum+=1*self.ws[m][-1]
            
            #filling the reseult into the lists that we considered for the result and net of the output layer neurones
            self.netm[m]=temp_sum
            self.om[m]=self.trans(self.netm[m])

    def iter_train(self,X,Y):
        #one iteration of training of the output layer using gradient decent
        self.forward(X)
        for m in range(self.M):
            delta=(Y[m]-self.om[m])*self.dtrans(self.netm[m])
            for k in range(self.K):
                wdif=self.eta*delta*self.rk[k]
                self.ws[m][k]=self.ws[m][k]-wdif
        
        
    def size_cal(self,centers,neighbors):
        #a function to calculate sizes of each rbf neurone
        
        # Avoid distance to itself
        distances = np.linalg.norm(centers[:, np.newaxis] - centers, axis=2)
        np.fill_diagonal(distances, np.inf)  
        sorted_distances = np.sort(distances, axis=1)
        sizes = np.mean(sorted_distances[:, :neighbors], axis=1)
        
        return sizes
    
    def train(self,X,Y,neighbors):
        
        #RBF layer train using Input Data driven method
        # Choose the number of RBF centers
        num_centers = self.K 
        X_np=np.array(X)
        initial_centers_indices = np.random.choice(range(len(X_np)), size=num_centers, replace=False).astype(int)
        centers = X_np[initial_centers_indices]
        sizes = self.size_cal(centers,neighbors)
        self.sk=sizes.tolist()

        #output layer training using gradient decent
        for i in range(len(X)):
            self.iter_train(X[i],Y[i])







Here are the settings for RBF layer and the input data.

In [324]:
input_path="PA-A_training_data_04.txt"
RBFCount=8
neighbors=3

The run and training section:
after setting all the variables next cell reads data from a file and start training.
Because of randomness each time that we run the program we get different results

In [325]:
X, y = [], []

with open(input_path, 'r') as inp:
    # the structure of the dataset is parsed from the second line
    inp.readline()
    in_n, out_n = inp.readline().split()[2:]
    in_n = int(in_n[2:])
    out_n = int(out_n[2:])

    for line in inp:
        if line and not line.startswith('#'):
            # we again, parse each line by a space to get the list of integer values
            vals = [int(float(v)) for v in line.strip('\n').strip(' ').split(' ') if v]

            # we split the values into x and y
            X.append(vals[:-out_n])
            y.append(vals[-out_n:])





#create the model and start training    
while(True):   
    try:
        rbf=RBF_net(in_n,RBFCount,out_n,False)
        rbf.train(X,y,neighbors)
        break
    except :
        print("overflow might happen or the number of the neurones inside rbf layers are more than P")
        time.sleep(2)
        print("if RBF layer setting is ok please wait!!")

#testing the model           
sum_error=0
for i in range(len(X)):
    rbf.forward(X[i])
        
    sum_error+=(rbf.om[0]-y[i][0])**2
avge=sum_error/len(y)
print("Done!")
print("error average:",avge)

Done!
error average: 0.2287941735899099
